In [0]:
spark

In [0]:
import os
import json

# Load kaggle.json (assuming it’s in the '/dbfs/tmp/' directory)
with open('/Workspace/Users/jkope@gmu.edu/kaggle.json', 'r') as f: # replace with your own path
    kaggle_credentials = json.load(f)

# Set the environment variables for authentication
os.environ['KAGGLE_USERNAME'] = kaggle_credentials['username']
os.environ['KAGGLE_KEY'] = kaggle_credentials['key']

# Check if credentials are set correctly
print("Kaggle credentials are set")


Kaggle credentials are set


In [0]:
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/82.7 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/78.2 kB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 6.5 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105786 sha256=1476e06c837092946b582ba7444f6ce340e85fa12a57a08147905d4ba17000e1
  Stored in directory: /root/.cache/pip/wheels/46/d2/26/84d0a1acdb9c6baccf7d28cf06962ec80529fe1ad938489983
Successfully built kaggle
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
!kaggle datasets download -d matt2434/cleaned-ecommerce-data -p /dbfs/tmp/kaggle_data && unzip /dbfs/tmp/kaggle_data/cleaned-ecommerce-data.zip -d /dbfs/tmp/kaggle_data


Dataset URL: https://www.kaggle.com/datasets/matt2434/cleaned-ecommerce-data
License(s): apache-2.0
cleaned-ecommerce-data.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /dbfs/tmp/kaggle_data/cleaned-ecommerce-data.zip
replace /dbfs/tmp/kaggle_data/2019-Dec-Cleaned.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:714)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:432)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:432)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.can

In [0]:
dbutils.fs.ls("dbfs:/tmp/kaggle_data")


[FileInfo(path='dbfs:/tmp/kaggle_data/2019-Dec-Cleaned.csv', name='2019-Dec-Cleaned.csv', size=395749939, modificationTime=1732248761000),
 FileInfo(path='dbfs:/tmp/kaggle_data/2019-Nov-Cleaned.csv', name='2019-Nov-Cleaned.csv', size=5981645247, modificationTime=1732248857000),
 FileInfo(path='dbfs:/tmp/kaggle_data/cleaned-ecommerce-data.zip', name='cleaned-ecommerce-data.zip', size=1776673282, modificationTime=1732248363000)]

In [0]:
nov_path = 'dbfs:/tmp/kaggle_data/2019-Nov-Cleaned.csv'
dec_path = 'dbfs:/tmp/kaggle_data/2019-Dec-Cleaned.csv'

novf = spark.read.csv(nov_path, header=True, inferSchema=True)
decf = spark.read.csv(dec_path, header=True, inferSchema=True)


In [0]:
novf.show()

+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|  brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|2019-11-01 00:00:00|      view|   1003461|2053013555631882655|electronics.smart...| xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|
|2019-11-01 00:00:00|      view|   5000088|2053013566100866035|appliances.sewing...| janome|293.65|530496790|8e5f4f83-366c-4f7...|
|2019-11-01 00:00:01|      view|   3601530|2053013563810775923|appliances.kitche...|     lg|712.87|518085591|3bfb58cd-7892-48c...|
|2019-11-01 00:00:01|      view|   1004775|2053013555631882655|electronics.smart...| xiaomi|183.27|558856683|313628f1-68b8-460...|
|2019-11-01 00:00:01|      view|   1306894|2053013558920217191|  computers.notebook

In [0]:
# Ensure schemas match (optional but recommended)
assert novf.schema == decf.schema, "DataFrames have different schemas!"

In [0]:
df = novf.union(decf)

In [0]:
# Original Code
#from pyspark.sql.functions import collect_list, col

# Filter for 'purchase' events only
#purchase_df = df.filter(col("event_type") == "purchase")

# Group by session and collect items purchased in each session
#transactions_df = purchase_df.groupBy("user_session").agg(collect_list("product_id").alias("items"))

# Show prepared data
#transactions_df.show(5, truncate=False)


com.databricks.backend.common.rpc.CommandSkippedException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:138)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:714)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:432)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:432)
	at com.databricks.spark.chauffeur.ExecutionContextManagerV1.cancelExecution(ExecutionContextManagerV1.scala:458)
	at com.databricks.spark.chauffeur.ChauffeurState.$anonfun$process$1(ChauffeurState.scala:537)
	at com.data

In [0]:
# Updated Code, Removes duplicate items
from pyspark.sql.functions import collect_list, col, array_distinct

# Filter for 'purchase' events only
purchase_df = df.filter(col("event_type") == "purchase")

# Group by session and collect unique items purchased in each session
transactions_df = purchase_df.groupBy("user_session").agg(array_distinct(collect_list("product_id")).alias("items"))

# Display prepared data
transactions_df.show(5, truncate=False)


+------------------------------------+----------+
|user_session                        |items     |
+------------------------------------+----------+
|0000c091-07d6-42b6-a7d8-75732b489429|[1005238] |
|0001a1b4-7adf-4c7a-9230-fdc1f1ae4e7c|[1004767] |
|0001d2b9-c9cc-474b-89d6-bf2db3ab2d66|[1005115] |
|0001f68e-6c17-42d2-9a04-f4d594c3eee3|[1307188] |
|00021444-7cf9-4256-8bdb-9abf9065ce34|[28722324]|
+------------------------------------+----------+
only showing top 5 rows



In [0]:
from pyspark.ml.fpm import FPGrowth

# Initialize and fit FP-Growth model
fp_growth = FPGrowth(itemsCol="items", minSupport=0.000075, minConfidence=0.05)  # Adjust parameters as needed
model = fp_growth.fit(transactions_df)

# Show frequent itemsets
print("Frequent Itemsets:")
model.freqItemsets.show(truncate=False)

# Show association rules
print("Association Rules:")
model.associationRules.show(truncate=False)

# Transform data (predict potential recommendations)
print("Predictions:")
model.transform(transactions_df).show(truncate=False)


Frequent Itemsets:
+------------------+-----+
|items             |freq |
+------------------+-----+
|[1005225]         |456  |
|[1002535]         |196  |
|[1306183]         |115  |
|[1003549]         |80   |
|[28719606]        |59   |
|[7005059]         |47   |
|[1004856]         |31294|
|[1307568]         |80   |
|[5100846]         |59   |
|[5800860]         |47   |
|[1004767]         |22184|
|[1004767, 1004856]|614  |
|[8801100]         |456  |
|[1002796]         |196  |
|[11200192]        |115  |
|[1005115]         |21828|
|[1005115, 1004767]|163  |
|[1005115, 1004856]|140  |
|[1004723]         |455  |
|[1005223]         |194  |
+------------------+-----+
only showing top 20 rows

Association Rules:
+------------------+----------+-------------------+------------------+---------------------+
|antecedent        |consequent|confidence         |lift              |support              |
+------------------+----------+-------------------+------------------+---------------------+
|[1004767